# CR ML Road Map


- Step 1:
    - TBA
- Step 2:
    - TBA 
- Step 3:
    - TBA

## Import Packages

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
# basic python package
import importlib
import numpy as np
import time
import logging
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# python ploting packages
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib import cm


# self-define classes
from script import CR_ML_Class as CR
from script import load_mock_data as LD


# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
print("Tensorflow Version is {}".format(tf.__version__))
print("Keras Version is {}".format(tf.keras.__version__))
!nvudia-smi

Tensorflow Version is 2.4.1
Keras Version is 2.4.0
/bin/bash: nvudia-smi: command not found


## Transfrom Mock Data from Text File to Numpy Dataframe 

- Data Description:


    - parameter : propagation and source parameters 
        numpy shape: (# of mock data, 14)
            #parameter: original parameter 
            #new_parameter: recalculate the normal factor and Ap 

            raw1=D_0, 
            raw2=\delta, 
            raw3=z_h, 
            raw4=v_A, 
            raw5=\eta, 
            raw6=A_p, 
            raw7=\nu_1, 
            raw8=\nu_2, 
            raw9=log10(R_{br,1}), 
            raw10=\nu_3, 
            raw11=log10(R_{br,2}), 
            raw12=N_{Li}, 
            raw13=N_{Be}, 
            raw14=N_{O}


    - data: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated parameters


     - chi: chi-square 
        numpy shape: (# of mock data)
            #new_chi: chi-square from "modify_data"

In [8]:
%%time
text_Data_path = "../Data/Text_Mock_data/"

mockdata_1 = CR.Mock_Data_to_NumpyArray(text_Data_path + "res_3x_1.txt")
parameter, data, chisq = mockdata_1.parameter, mockdata_1.spectrum, mockdata_1.chisq

logging.info("Data Shape for 'parameter': {}".format(parameter.shape))
logging.info("Data Shape for 'data': {}".format(data.shape))
logging.info("Data Shape for 'chisq': {}".format(chisq.shape))

INFO:root:Thu Aug 05 14:41:34 2021
INFO:root:Now loading...
INFO:root:Total data: 52836
INFO:root:Time consumption : 0.7570 min
INFO:root:Data Shape for 'parameter': (52836, 14)
INFO:root:Data Shape for 'data': (4438224, 11)
INFO:root:Data Shape for 'chisq': (52836,)


CPU times: user 44.7 s, sys: 1.47 s, total: 46.1 s
Wall time: 46.1 s


## ReCalculate Normalilzed Factor for Mock Data

- 1: Using 
    ```
    CR.Mock_Data_Rescale(parameter, parameter, data)
    ```
    to split 'data' into Li, Be, B, C and O `spectra`.  
    Note that we put the same 'parameter' here because we have not get new normalilzed factor yet
    
    - `spectra`: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated paramete

In [11]:
%%time
spectra_data = CR.Mock_Data_Rescale(parameter, parameter, data)
spectra_data = spectra_data.data
logging.info("Data Shape for 'spectra_data': {}".format(spectra_data.shape))
logging.info("There are {} mock data.".format(spectra_data.shape[0]))
logging.info("For each mock data, there are {} energy bins.".format(spectra_data.shape[1]))
logging.info("{} corresponding to E, Li, Be, B, C and O.".format(spectra_data.shape[2]))

INFO:root:Data Shape for 'spectra_data': (52836, 84, 6)
INFO:root:There are 52836 mock data.
INFO:root:For each mock data, there are 84 energy bins.
INFO:root:6 corresponding to E, Li, Be, B, C and O.


CPU times: user 338 ms, sys: 17.8 ms, total: 356 ms
Wall time: 356 ms


In [57]:
%%time
importlib.reload(CR)
new_Ap = CR.ReCalculateAp(spectra_data[:1000]) 
new_Ap.GetBestAp()

INFO:root:Thu Aug 05 17:04:47 2021
INFO:root:Finding best Ap
INFO:root:=====START=====
100%|██████████| 1000/1000 [00:03<00:00, 267.77it/s]
INFO:root: Time Cost for this Step : 0.0623 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0625 min


CPU times: user 3.77 s, sys: 15.8 ms, total: 3.79 s
Wall time: 3.78 s


In [32]:
from scipy import interpolate

logging.info(time.strftime("%a %b %d %H:%M:%S %Y", time.localtime()))
ticks_1 = time.time()
#######################################################################################################

"""
Load Experimental Data
"""
####################################################################################
exp_data_path = "../Data/Exp_Data/"

LiAMS, LiV = np.load(exp_data_path + "Li_AMS2.npy"),np.load(exp_data_path + "Li_Voyager.npy")
Li_Eams, Li_Fams, Li_Sams = LiAMS[0], LiAMS[1], LiAMS[2]
Li_Evy, Li_Fvy, Li_Svy = LiV[0], LiV[1], LiV[2]

BeAMS, BeV = np.load(exp_data_path + "Be_AMS2.npy"), np.load(exp_data_path + "Be_Voyager.npy")
Be_Eams, Be_Fams, Be_Sams = BeAMS[0], BeAMS[1], BeAMS[2]
Be_Evy, Be_Fvy, Be_Svy = BeV[0], BeV[1], BeV[2]

BAMS, BV, BA = np.load(exp_data_path + "B_AMS2.npy"), np.load(exp_data_path + "B_Voyager.npy"), np.load(exp_data_path + "B_ACE.npy")
B_Eams, B_Fams, B_Sams = BAMS[0], BAMS[1], BAMS[2]
B_Evy, B_Fvy, B_Svy = BV[0], BV[1], BV[2]
B_Eace,B_Face,B_Sace = BA[0], BA[1], BA[2]

CAMS, CV, CA = np.load(exp_data_path + "C_AMS2.npy"), np.load(exp_data_path + "C_Voyager.npy"), np.load(exp_data_path + "C_ACE.npy")
C_Eams, C_Fams, C_Sams = CAMS[0], CAMS[1], CAMS[2]
C_Evy, C_Fvy, C_Svy = CV[0], CV[1], CV[2]
C_Eace,C_Face,C_Sace = CA[0], CA[1], CA[2]

OAMS, OV, OA = np.load(exp_data_path + "O_AMS2.npy"), np.load(exp_data_path + "O_Voyager.npy"), np.load(exp_data_path + "O_ACE.npy")
O_Eams, O_Fams, O_Sams = OAMS[0], OAMS[1], OAMS[2]
O_Evy, O_Fvy, O_Svy = OV[0], OV[1], OV[2]
O_Eace,O_Face,O_Sace = OA[0], OA[1], OA[2]
####################################################################################

def findbestAp(E,Li,Be,B,C,O):
    aplist = np.linspace(1.9135/5.,7.0114/5.,120)
    chisq = []
    for ap in aplist:     
        norm_interpolate_Li = interpolate.interp1d((E), (Li*ap), kind='cubic')
        norm_interpolate_Be = interpolate.interp1d((E), (Be*ap), kind='cubic')
        pred_interpolate_B  = interpolate.interp1d((E), (B*ap), kind='cubic')
        pred_interpolate_C  = interpolate.interp1d((E), (C*ap), kind='cubic')
        norm_interpolate_O  = interpolate.interp1d((E), (O*ap), kind='cubic')
        chisq.append(     np.sum((norm_interpolate_Li(Li_Eams)-np.array(Li_Fams))**2/np.array(Li_Sams)**2)+
                          np.sum((norm_interpolate_Li(Li_Evy)-np.array(Li_Fvy))**2/np.array(Li_Svy)**2)+

                          np.sum((norm_interpolate_Be(Be_Eams)-np.array(Be_Fams))**2/np.array(Be_Sams)**2)+
                          np.sum((norm_interpolate_Be(Be_Evy)-np.array(Be_Fvy))**2/np.array(Be_Svy)**2)+

                          np.sum((pred_interpolate_B(B_Eams)-np.array(B_Fams))**2/np.array(B_Sams)**2)+
                          np.sum((pred_interpolate_B(B_Evy)-np.array(B_Fvy))**2/np.array(B_Svy)**2)+
                          np.sum((pred_interpolate_B(B_Eace)-np.array(B_Face))**2/np.array(B_Sace)**2)+

                          np.sum((pred_interpolate_C(C_Eams)-np.array(C_Fams))**2/np.array(C_Sams)**2)+
                          np.sum((pred_interpolate_C(C_Evy)-np.array(C_Fvy))**2/np.array(C_Svy)**2)+
                          np.sum((pred_interpolate_C(C_Eace)-np.array(C_Face))**2/np.array(C_Sace)**2)+

                          np.sum((norm_interpolate_O(O_Eams)-np.array(O_Fams))**2/np.array(O_Sams)**2)+
                          np.sum((norm_interpolate_O(O_Evy)-np.array(O_Fvy))**2/np.array(O_Svy)**2)+
                          np.sum((norm_interpolate_O(O_Eace)-np.array(O_Face))**2/np.array(O_Sace)**2)
                         )
    return chisq.index(min(chisq)), chisq

INFO:root:Thu Aug 05 16:52:04 2021


In [33]:
%%time
aaa = findbestAp(E[0],Li[0],Be[0],B[0],C[0],O[0])

CPU times: user 239 ms, sys: 3.01 ms, total: 242 ms
Wall time: 240 ms


In [40]:
from scipy import interpolate
def interpolate_chisq(P,F,S):
    return np.round(np.sum(((P-np.array(F))**2)/(np.array(S)**2), axis=0), 10)
E, Li, Be, B, C, O = spectra_data[:,:,0], spectra_data[:,:,1],spectra_data[:,:,2],spectra_data[:,:,3],spectra_data[:,:,4],spectra_data[:,:,5]

aplist = np.linspace(1.9135/5.,7.0114/5.,120)
norm_interpolate_Li = interpolate.interp1d((E[0]), (Li[0]*aplist[0]), kind='cubic')

print(interpolate_chisq(norm_interpolate_Li(Li_Eams),Li_Fams,Li_Sams))

24282.9517446915


In [41]:
%%time
def interpolate_chisq(P,F,S):
    return np.round(np.sum(((P-np.array(F))**2)/(np.array(S)**2), axis=1), 10)
E, Li, Be, B, C, O = spectra_data[:,:,0], spectra_data[:,:,1],spectra_data[:,:,2],spectra_data[:,:,3],spectra_data[:,:,4],spectra_data[:,:,5]

aplist = np.linspace(1.9135/5.,7.0114/5.,120).reshape(120,1)
norm_interpolate_Li = interpolate.interp1d((E[0]), (Li[0]*aplist), kind='cubic')
norm_interpolate_Be = interpolate.interp1d((E[0]), (Be[0]*aplist), kind='cubic')
pred_interpolate_B  = interpolate.interp1d((E[0]), (B[0]*aplist), kind='cubic')
pred_interpolate_C  = interpolate.interp1d((E[0]), (C[0]*aplist), kind='cubic')
norm_interpolate_O  = interpolate.interp1d((E[0]), (O[0]*aplist), kind='cubic')

chisq = (interpolate_chisq(norm_interpolate_Li(Li_Eams),Li_Fams,Li_Sams)+
        interpolate_chisq(norm_interpolate_Li(Li_Evy),Li_Fvy,Li_Svy)+
    
        interpolate_chisq(norm_interpolate_Be(Be_Eams),Be_Fams,Be_Sams)+
        interpolate_chisq(norm_interpolate_Be(Be_Evy),Be_Fvy,Be_Svy)+
        
        interpolate_chisq(pred_interpolate_B(B_Eams),B_Fams,B_Sams)+
        interpolate_chisq(pred_interpolate_B(B_Evy),B_Fvy,B_Svy)+
        interpolate_chisq(pred_interpolate_B(B_Eace),B_Face,B_Sace)+
        
        interpolate_chisq(pred_interpolate_C(C_Eams),C_Fams,C_Sams)+
        interpolate_chisq(pred_interpolate_C(C_Evy),C_Fvy,C_Svy)+
        interpolate_chisq(pred_interpolate_C(C_Eace),C_Face,C_Sace)+
        
        interpolate_chisq(norm_interpolate_O(O_Eams),O_Fams,O_Sams)+
        interpolate_chisq(norm_interpolate_O(O_Evy),O_Fvy,O_Svy)+
        interpolate_chisq(norm_interpolate_O(O_Eace),O_Face,O_Sace))
        
        
        

print(interpolate_chisq(norm_interpolate_Li(Li_Eams),Li_Fams,Li_Sams).shape)
print(chisq.shape)

(120,)
(120,)
CPU times: user 11.1 ms, sys: 76 µs, total: 11.2 ms
Wall time: 9.55 ms


In [50]:
aplist[np.where(chisq == min(chisq))[0]]

array([[0.99958874]])